# Import packages

In [11]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import numpy as np

# Database connection

In [2]:
engine = create_engine('sqlite:///movie-lens.db', echo = True)
conn = engine.connect()

# User-based collaborative filtering using the Pearson correlation function

In [31]:
def calculate_similarities(user_a_id):
    # Get mean rating of user a
    user_a_mean_rating_query = text(f'SELECT AVG(rating) FROM ratings WHERE userId = {user_a_id}')
    user_a_mean_rating_result = conn.execute(user_a_mean_rating_query)
    user_a_mean_rating = user_a_mean_rating_result.fetchone()[0]
    # Get all other user b IDs who are in 'ratings' table
    user_b_ids_query = text(f'SELECT DISTINCT userId FROM ratings WHERE userId != {user_a_id}')
    user_b_ids_result = conn.execute(user_b_ids_query)
    user_b_ids = np.array(user_b_ids_result.fetchall()).transpose()[0]
    similarities = [] # Similarity between the user a and every user b
    for user_b_id in user_b_ids:
        # Get rating of user a and user b for each movie which is rated by both users
        both_users_ratings_each_movie_query = text(f'SELECT r1.movieId, r1.userId, r1.rating, r2.userId, r2.rating FROM ratings r1, ratings r2 WHERE r1.userId = {user_a_id} AND r2.userId = {user_b_id} AND r1.movieId = r2.movieId')
        both_users_ratings_each_movie_result = conn.execute(both_users_ratings_each_movie_query)
        both_users_ratings_each_movie = both_users_ratings_each_movie_result.fetchall()
        if both_users_ratings_each_movie.__len__() == 0:
            continue
        _ , _, user_a_rating_list, _, user_b_rating_list = zip(*both_users_ratings_each_movie)
        user_a_rating_list = np.array(user_a_rating_list)
        user_b_rating_list = np.array(user_b_rating_list)
        # Get mean rating of user b
        user_b_mean_rating_query = text(f'SELECT AVG(rating) FROM ratings WHERE userId = {user_b_id}')
        user_b_mean_rating_result = conn.execute(user_b_mean_rating_query)
        user_b_mean_rating = user_b_mean_rating_result.fetchone()[0]
        # Calculate similarity between user a and b
        numerator = np.sum(np.multiply(user_a_rating_list - user_a_mean_rating, user_b_rating_list - user_b_mean_rating))
        denominator = np.sqrt(np.sum(np.multiply(user_a_rating_list - user_a_mean_rating, user_a_rating_list - user_a_mean_rating))) * np.sqrt(np.sum(np.multiply(user_b_rating_list - user_b_mean_rating, user_b_rating_list - user_b_mean_rating)))
        sim_a_b = numerator / denominator
        similarities.append([user_b_id, sim_a_b])
    similarities.sort(key=lambda sim: sim[1], reverse=True) # Sort by similarity values, descending
    return similarities

In [32]:
sim_list = calculate_similarities(1)
sim_list

2025-10-25 19:37:32,665 INFO sqlalchemy.engine.Engine SELECT AVG(rating) FROM ratings WHERE userId = 1
2025-10-25 19:37:32,668 INFO sqlalchemy.engine.Engine [cached since 2562s ago] ()
2025-10-25 19:37:32,679 INFO sqlalchemy.engine.Engine SELECT DISTINCT userId FROM ratings WHERE userId != 1
2025-10-25 19:37:32,680 INFO sqlalchemy.engine.Engine [cached since 1.125e+04s ago] ()
2025-10-25 19:37:32,703 INFO sqlalchemy.engine.Engine SELECT r1.movieId, r1.userId, r1.rating, r2.userId, r2.rating FROM ratings r1, ratings r2 WHERE r1.userId = 1 AND r2.userId = 2 AND r1.movieId = r2.movieId
2025-10-25 19:37:32,703 INFO sqlalchemy.engine.Engine [cached since 9693s ago] ()
2025-10-25 19:37:32,728 INFO sqlalchemy.engine.Engine SELECT AVG(rating) FROM ratings WHERE userId = 2
2025-10-25 19:37:32,728 INFO sqlalchemy.engine.Engine [cached since 414.3s ago] ()
2025-10-25 19:37:32,737 INFO sqlalchemy.engine.Engine SELECT r1.movieId, r1.userId, r1.rating, r2.userId, r2.rating FROM ratings r1, ratings r

C:\Users\minhn\AppData\Local\Temp\ipykernel_1460\1899846935.py:28: RuntimeWarning: invalid value encountered in scalar divide
  sim_a_b = numerator / denominator


2025-10-25 19:37:34,697 INFO sqlalchemy.engine.Engine [cached since 414.3s ago] ()
2025-10-25 19:37:34,710 INFO sqlalchemy.engine.Engine SELECT r1.movieId, r1.userId, r1.rating, r2.userId, r2.rating FROM ratings r1, ratings r2 WHERE r1.userId = 1 AND r2.userId = 60 AND r1.movieId = r2.movieId
2025-10-25 19:37:34,710 INFO sqlalchemy.engine.Engine [cached since 247.8s ago] ()
2025-10-25 19:37:34,736 INFO sqlalchemy.engine.Engine SELECT AVG(rating) FROM ratings WHERE userId = 60
2025-10-25 19:37:34,736 INFO sqlalchemy.engine.Engine [cached since 414.3s ago] ()
2025-10-25 19:37:34,747 INFO sqlalchemy.engine.Engine SELECT r1.movieId, r1.userId, r1.rating, r2.userId, r2.rating FROM ratings r1, ratings r2 WHERE r1.userId = 1 AND r2.userId = 61 AND r1.movieId = r2.movieId
2025-10-25 19:37:34,747 INFO sqlalchemy.engine.Engine [cached since 247.8s ago] ()
2025-10-25 19:37:34,770 INFO sqlalchemy.engine.Engine SELECT AVG(rating) FROM ratings WHERE userId = 61
2025-10-25 19:37:34,771 INFO sqlalchem

[[77, 1.0000000000000002],
 [12, 1.0],
 [85, 1.0],
 [253, 1.0],
 [291, 1.0],
 [358, 1.0],
 [388, 1.0],
 [2, 0.9999999999999998],
 [146, 0.9990496408681655],
 [278, 0.9710607611177227],
 [550, 0.9500648497978605],
 [13, 0.9478788458420679],
 [127, 0.9406906663964631],
 [333, 0.9379233588635404],
 [472, 0.9293520860459321],
 [157, 0.901774575834698],
 [139, 0.8903416876712336],
 [401, 0.8713212630061857],
 [511, 0.8655816064894617],
 [473, 0.840746829564101],
 [366, 0.8352756122978426],
 [258, 0.8320502943378436],
 [487, 0.8220948302697891],
 [90, 0.8214224137571167],
 [180, 0.812323733591591],
 [499, 0.8009893836412011],
 [430, 0.792673331599083],
 [207, 0.7722519995654104],
 [535, 0.7465137267234351],
 [114, 0.7439847032746658],
 [154, 0.7122202243459141],
 [162, 0.6939887258055054],
 [210, 0.6842827829019166],
 [398, 0.6657357370608938],
 [505, 0.6581927982722249],
 [324, 0.6507913734559686],
 [206, 0.6378635032837362],
 [478, 0.6368783518755518],
 [369, 0.6281219378489326],
 [421, 0.

# Close the connection

In [33]:
conn.close()

2025-10-25 19:40:21,872 INFO sqlalchemy.engine.Engine ROLLBACK
